# config

In [4]:
from getpass import getpass
import requests, json

token = getpass("Enter GitHub token (hidden): ")

r = requests.get("https://api.github.com/user", headers={"Authorization": f"token {token}"})
print(r.status_code)
print(r.json().get("login"))

Enter GitHub token (hidden): ··········
200
Vu-Quoc-Tuan


In [5]:
username = "Vu-Quoc-Tuan"
repo_name = "BTL-NLP-2526I_INT3406_3"

# Sử dụng biến token để xác thực trực tiếp trong URL
!git clone https://{token}@github.com/{username}/{repo_name}

Cloning into 'BTL-NLP-2526I_INT3406_3'...
remote: Enumerating objects: 232, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 232 (delta 26), reused 41 (delta 18), pack-reused 177 (from 1)
Receiving objects: 100% (232/232), 136.89 MiB | 16.18 MiB/s, done.
Resolving deltas: 100% (107/107), done.
Updating files: 100% (49/49), done.


In [6]:
# 1) vào thư mục repo (thay path nếu khác)
%cd /content/BTL-NLP-2526I_INT3406_3

# 2) kiểm tra hiện tại đang ở đâu và liệt kê file
!pwd
!ls -la
!ls -la vlsp-mt
!test -f vlsp-mt/requirements.txt && echo "requirements found" || echo "requirements NOT found"

# 3) show python interpreter đang dùng
import sys
print("Python executable:", sys.executable)

# 4) dùng chính interpreter đó để cài (an toàn hơn dùng !pip trực tiếp)
!{sys.executable} -m pip install -r vlsp-mt/requirements.txt

/content/BTL-NLP-2526I_INT3406_3
/content/BTL-NLP-2526I_INT3406_3
total 28
drwxr-xr-x 5 root root 4096 Dec 22 11:26 .
drwxr-xr-x 1 root root 4096 Dec 22 11:26 ..
drwxr-xr-x 8 root root 4096 Dec 22 11:26 .git
-rw-r--r-- 1 root root   60 Dec 22 11:26 .gitignore
-rw-r--r-- 1 root root   25 Dec 22 11:26 README.md
drwxr-xr-x 4 root root 4096 Dec 22 11:26 transformer_base
drwxr-xr-x 4 root root 4096 Dec 22 11:26 vlsp-mt
total 80
drwxr-xr-x 4 root root  4096 Dec 22 11:26 .
drwxr-xr-x 5 root root  4096 Dec 22 11:26 ..
drwxr-xr-x 6 root root  4096 Dec 22 11:26 data
-rw-r--r-- 1 root root 39689 Dec 22 11:26 REPORT.md
-rw-r--r-- 1 root root   485 Dec 22 11:26 requirements.txt
-rw-r--r-- 1 root root 11658 Dec 22 11:26 RUN_GUIDE.md
-rw-r--r-- 1 root root  4621 Dec 22 11:26 run.md
drwxr-xr-x 3 root root  4096 Dec 22 11:26 scripts
requirements found
Python executable: /usr/bin/python3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Run script

## Preprocessing

In [7]:
%cd vlsp-mt

/content/BTL-NLP-2526I_INT3406_3/vlsp-mt


## Train

In [8]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.2 MB/s eta 0:00:00


In [9]:
!python scripts/train_qwen_lora.py \
  --model_name Qwen/Qwen2.5-3B-Instruct \
  --direction en2vi \
  --src data/dedup/train.en \
  --tgt data/dedup/train.vi \
  --val_src data/clean/dev.en \
  --val_tgt data/clean/dev.vi \
  --run_id en2vi_v1 \
  --epochs 2 \
  --batch_size 24 \
  --grad_accum 2 \
  --lr 1.5e-4 \
  --lora_r 32 \
  --lora_alpha 64 \
  --neftune_alpha 3.0 \
  --eval_bleu \
  --bleu_sample_size 100 \
  --eval_steps 3000 \
  --early_stopping_patience 3 \
  --no_grad_checkpoint \
  --max_len 256 \
  --medical_vocab data/medical_vocab.txt \
  --init_new_embeddings_avg

2025-12-22 11:26:40.761667: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-22 11:26:40.779464: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766402800.801291    6583 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766402800.807952    6583 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766402800.824504    6583 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

## Eval

In [2]:
## Public test

In [34]:
!python scripts/generate.py \
    --model_name Qwen/Qwen2.5-3B-Instruct \
    --adapter_path runs/en2vi_v1/lora_en2vi_sft \
    --direction en2vi \
    --input data/raw/public_test.en.txt \
    --output outputs/public_test_en2vi_sft.hyp.vi \
    --batch_size 24 \
    --num_beams 5 \
    --repetition_penalty 1.2

2025-12-22 19:32:09.960918: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-22 19:32:09.979297: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766431930.001610  126631 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766431930.008239  126631 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766431930.025273  126631 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [35]:
!python scripts/eval_bleu.py \
    --hyp outputs/public_test_en2vi_sft.hyp.vi \
    --ref data/raw/public_test.vi.txt \
    --src data/raw/public_test.en.txt \
    --show_worst 10

Loading hypothesis: outputs/public_test_en2vi_sft.hyp.vi
Loading reference: data/raw/public_test.vi.txt
Loading source: data/raw/public_test.en.txt

Evaluating 3000 sentence pairs...

EVALUATION RESULTS
  BLEU:    52.78
  chrF++:  66.13
  chrF:    66.26
  TER:     40.87 (lower is better)
  METEOR:  74.84

10 WORST TRANSLATIONS (by BLEU):
--------------------------------------------------

[1] Line 2879 | BLEU: 3.8 | chrF: 16.1
  SRC: Severe respiratory failure was observed on 56% of PHOCD and on 27% of PHCD.
  HYP: 56% trẻ TAP nặng và 27% trẻ TAP không nặng có suy thông khí mức độ nặng.
  REF: Khởi phát bệnh sớm và biểu hiện suy hô hấp nặng ở nhóm TAP không liên quan tim cao hơn nhóm TAP liên quan tim (56% so với 27%).

[2] Line 1858 | BLEU: 4.9 | chrF: 30.0
  SRC: 4 / No side effects are found in patients using Hairy antler.
  HYP: 4 / Không có tác dụng không mong muốn nào được ghi nhận ở các bệnh nhân sử dụng Nhung nai.
  REF: Chưa phát hiện tác dụng phụ khi dùng viên nhung nai.

[3]

In [3]:
## Test

In [30]:
!python scripts/generate.py \
    --model_name Qwen/Qwen2.5-3B-Instruct \
    --adapter_path runs/en2vi_v1/lora_en2vi_sft \
    --direction en2vi \
    --input data/clean/test.en \
    --output outputs/test_en2vi.hyp.vi \
    --batch_size 16 \
    --num_beams 4 \
    --repetition_penalty 1.1

2025-12-22 18:33:40.397806: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-22 18:33:40.416245: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766428420.438876  111891 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766428420.445623  111891 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766428420.463156  111891 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [31]:
!python scripts/eval_bleu.py \
    --hyp outputs/test_en2vi.hyp.vi \
    --ref data/clean/test.vi \
    --src data/clean/test.en \
    --show_worst 10

Loading hypothesis: outputs/test_en2vi.hyp.vi
Loading reference: data/clean/test.vi
Loading source: data/clean/test.en

Evaluating 1000 sentence pairs...

EVALUATION RESULTS
  BLEU:    50.76
  chrF++:  64.92
  chrF:    64.99
  TER:     41.70 (lower is better)
  METEOR:  74.17

10 WORST TRANSLATIONS (by BLEU):
--------------------------------------------------

[1] Line 499 | BLEU: 1.8 | chrF: 3.2
  SRC: KNOWLEGE OF DISEASES IN SCHOOL, SOCIAL VICES AND BAD HOBITS OF 10TH GRADE STUDENTS IN nAMdINH PROVINCE IN 2013
  HYP: Kiến thức về bệnh tật học đường, tệ nạn xã hội và thói quen xấu của học sinh lớp 10 tại tỉnh Nam Định năm 2013
  REF: NHẬN THỨC VỀ MỘT SỐ BỆNH TẬT HỌC ĐƯỜNG, TỆ NẠN XÃ HỘI, THÓI QUEN XẤU CỦA HỌC SINH LỚP 10 TẠI NAM ĐỊNH NĂM 2013

[2] Line 860 | BLEU: 1.9 | chrF: 1.4
  SRC: RESULT OF TREATMENT OF SEVERE PNEUMONIA IN CHILDREN FROM 1 MONTH TO FIVE YEARS OLD IN QUANG NINH PROVINC
  HYP: KẾT QUẢ ĐIỀU TRỊ VIÊM PHỔI NẶNG Ở TRẺ EM TỪ 1 THÁNG ĐẾN 5 TUỔI TẠI TỈNH QUẢNG NINH
  REF:

In [4]:
# Test unseen

In [41]:
!python scripts/generate.py \
    --model_name Qwen/Qwen2.5-3B-Instruct \
    --adapter_path runs/en2vi_v1/lora_en2vi_sft \
    --direction en2vi \
    --input data/raw/test_unseen_v3.en.txt \
    --output outputs/test_unseen_v3.hyp.vi \
    --batch_size 16 \
    --num_beams 4 \
    --repetition_penalty 1.1

2025-12-22 20:54:06.954644: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-22 20:54:06.972270: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766436846.994049  147099 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766436847.000536  147099 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766436847.017108  147099 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

data/raw/test_unseen_v3.en.txt

In [42]:
!python scripts/eval_bleu.py \
    --hyp outputs/test_unseen_v3.hyp.vi \
    --ref data/raw/test_unseen_v3.vi.txt \
    --src data/raw/test_unseen_v3.en.txt \
    --show_worst 10

Loading hypothesis: outputs/test_unseen_v3.hyp.vi
Loading reference: data/raw/test_unseen_v3.vi.txt
Loading source: data/raw/test_unseen_v3.en.txt

Evaluating 1152 sentence pairs...

EVALUATION RESULTS
  BLEU:    50.15
  chrF++:  64.16
  chrF:    64.33
  TER:     43.38 (lower is better)
  METEOR:  73.20

10 WORST TRANSLATIONS (by BLEU):
--------------------------------------------------

[1] Line 492 | BLEU: 2.8 | chrF: 7.8
  SRC: Pherochromocytomas in children - Genotypic and phenotype
  HYP: Pherochromocytomas in children - Genotypic and phenotype
  REF: U tuỷ thượng thận ở trẻ em - Kiểu gen và kiểu hình

[2] Line 847 | BLEU: 4.1 | chrF: 27.7
  SRC: Hemorrhagic strokes consist of a highest rate 86, 7% (26/30 patients), transient ischemic attack rate is 10% (3/30 patients), neurological deficits rate is 3, 3% (1/30 patients) Conclusions: Moyamoya disease is a chronic cerebrovascular occlusive disease with a long history carrying risk of either ischemic or haemorrhage events. Revascula